# Programming Assignment 1
## model transformers:
### 4. Semistructured to Property Graph
Each of the six groups should choose one and implement it in Python with a README file and some documentation. The implementation can be delivered in one of three ways: a command line utility, a Python package, or a REST API. A model transformer will take as input (a) a data file (e.g., CSV) of a specific model, and (b) a mapping configuration (in class we have seen several examples -- see the Model Transformation slides) and output a transformed data file. The developing group should also produce a sample input data file and its corresponding output data file. If any implementation uses an existing library or has some dependency (e.g., it uses Java), it should be mentioned in the README file so that the users can configure their environment accordingly. 

In [49]:
#pip install jsonpath-ng
import json
from jsonpath_ng.ext import parse

In [50]:
# Load people.json as Data Input-I
#people.json
with open('./people2.json') as people_json:
    people = json.load(people_json)

people

{'People': [{'id': 1,
   'Name': 'Max',
   'Age': 20,
   'Gender': 'Male',
   'friends': [2, 3]},
  {'id': 2, 'Name': 'Alice', 'Age': 25, 'Gender': 'Female', 'friends': [1]},
  {'id': 3, 'Name': 'Bob', 'Age': 22, 'Gender': 'Male', 'friends': [1, 4]},
  {'id': 4, 'Name': 'Joe', 'Age': 52, 'Gender': 'Male', 'friends': [3]},
  {'id': 5, 'Name': 'Van Dyke', 'Age': 42, 'Gender': '', 'friends': [2]},
  {'id': 6,
   'Name': ['Ken', 'Kennedy', 'Kenneth'],
   'Age': 'unknown',
   'Gender': 'Male',
   'friends': []},
  {'id': 7,
   'Name': ['Sally'],
   'Age': '19',
   'Gender': 'Female',
   'friends': [2, 1, 3, 6, 5, 4]},
  {'id': 8,
   'Name': ['Ka‘anapali'],
   'Age': '19',
   'Gender': 'Not Stated',
   'friends': [5, 4]}]}

In [51]:
# Exploration : Find People Gender where Name conatins Max 
jsonpath_expression = parse("$.People[?(@.Name[:] =~ 'Max')].Gender")

for match in jsonpath_expression.find(people):
    print(match.value)

Male


In [52]:
import json

# Load the source data from a JSON Source file
with open('./people2.json') as people_json:
    people = json.load(people_json)

source_data=people
# Load the mapping configuration
with open("test_person_schema_map.json", "r") as mapping_file:
    mapping_config = json.load(mapping_file)

# Initialize the property graph
property_graph = {
    "graph": {
        "nodes": [],
        "edges": []
    }
}

# add nodes to the property graph
for element in source_data['People']:
    node = {
        "node": {
            "id": element['id'],
            "label": mapping_config[0]['schema-map']['elements'][0]['label'],
            "properties": {}
        }
    }

    # Map properties to the node
    for property_name, property_mapping in mapping_config[0]['schema-map']['elements'][0]['node-properties'].items():
        property_value = element
        for key in property_mapping.split('.'):
            property_value = property_value.get(key, None)
            if property_value is None:
                break

        if property_value is not None:
            node["node"]["properties"][property_name] = {
                "datatype": "string",  # comment : modify this based on the actual data types
                "data value": property_value
            }

    property_graph["graph"]["nodes"].append(node)

# source data and add edges to the property graph
for element in source_data['People']:
    source_id = element['id']
    friends = element.get('friends', [])

    for friend_id in friends:
        edge = {
            "edge": {
                "id": len(property_graph["graph"]["edges"]) + 1,
                "relationship": mapping_config[1]['schema-map']['elements'][0]['relationship'],
                "from_node_id": source_id,
                "to_node_id": friend_id,
                "properties": {}
            }
        }

        property_graph["graph"]["edges"].append(edge)

# Save the property graph in JSON format
with open("property_graph.json", "w") as output_file:
    json.dump(property_graph, output_file, indent=4)

property_graph


{'graph': {'nodes': [{'node': {'id': 1, 'label': 'People', 'properties': {}}},
   {'node': {'id': 2, 'label': 'People', 'properties': {}}},
   {'node': {'id': 3, 'label': 'People', 'properties': {}}},
   {'node': {'id': 4, 'label': 'People', 'properties': {}}},
   {'node': {'id': 5, 'label': 'People', 'properties': {}}},
   {'node': {'id': 6, 'label': 'People', 'properties': {}}},
   {'node': {'id': 7, 'label': 'People', 'properties': {}}},
   {'node': {'id': 8, 'label': 'People', 'properties': {}}}],
  'edges': [{'edge': {'id': 1,
     'relationship': 'FRIENDS',
     'from_node_id': 1,
     'to_node_id': 2,
     'properties': {}}},
   {'edge': {'id': 2,
     'relationship': 'FRIENDS',
     'from_node_id': 1,
     'to_node_id': 3,
     'properties': {}}},
   {'edge': {'id': 3,
     'relationship': 'FRIENDS',
     'from_node_id': 2,
     'to_node_id': 1,
     'properties': {}}},
   {'edge': {'id': 4,
     'relationship': 'FRIENDS',
     'from_node_id': 3,
     'to_node_id': 1,
     'pro